<a href="https://colab.research.google.com/github/phisan-chula/Construction_Survey/blob/main/Geod_ThaiDatum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thai Datum Transformation via pyproj using 'pipeline' technique 
for concise and clear calculation step.<br>
Example from the Roytal Thai Irrigation Department<br>
QR Code GPS Benchmark Description<br>
<br>
 Phisan Santitamnont, Chulalongkorn University<br>
 Oct 22, 2022<br>
 <br>

![](https://raw.githubusercontent.com/phisan-chula/Construction_Survey/main/QR_Code_RID_61032x.png)

In [95]:
! pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


The 'pipeline' for datum tranformation could be generated by "proj4/projinfo" command in different formats eg.  PROJ,WKT_ALL,WKT2:2015,WKT2:2019,WKT1:GDAL,WKT1:ESRI,PROJJSON <br>
Change 0 or 1 in the cell below<br>
TEST_PROJINFO=1   ... to test projinfo ....<br>
TEST_PROJINFO=0   ... to skip this long process<br>

In [96]:
%%shell
TEST_PROJINFO=0
if [ $TEST_PROJINFO -eq 1 ]; then
    echo "@@@@@@ Preparing proj4 .... @@@@@@"
    conda install -c conda-forge proj
    conda install -c conda-forge proj-data
    echo "@@@@@@ Testing projinfo  ... @@@@@@"
    proj -v
    projinfo -o PROJ  -s  EPSG:4326  -t  EPSG:4240
    projinfo -o PROJ -s EPSG:32647 -t  EPSG:24047
fi;

In [97]:
! pip3 install pyproj
! pip3 install pygeodesy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
from pyproj import Transformer, Proj
import numpy as np
import pygeodesy as pgd
import pandas as pd
pd.options.display.max_colwidth = 100

#############################################################
def dd2DMS( dd, PREC=5, POS=''  ):
    '''conver degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )
_dd2DMS = np.vectorize( dd2DMS )

#############################################################
def DuoGrid_AziDist(DUO, COL_N, COL_E ):
    duoEN = DUO[[COL_N,COL_E]].to_numpy()
    dist = np.linalg.norm( duoEN[:1,]-duoEN[0:,])
    azim = np.arctan2( duoEN[1,1]-duoEN[0,1], duoEN[1,0]-duoEN[0,0] )
    _, azim = divmod( np.degrees(azim), 360 )
    duo_name = f'{DUO.iloc[0].Name:}->{DUO.iloc[1].Name:}'
    print( f'{duo_name:^20s} grid : azi = {dd2DMS(azim,PREC=2):}   dist = {dist:10.3f} meter')

#############################################################

RID_SHIFT = ' +x=204.4798 +y=837.8940 +z=294.7765 '
# adapting from  ! projinfo -o PROJ -s EPSG:32647 -t  EPSG:24047
UTMz47_W84toI75 =  '''+proj=pipeline  
  +step +inv +proj=utm +zone=47 +ellps=WGS84 
  +step +proj=push +v_3 
  +step +proj=cart +ellps=WGS84 
  +step +inv +proj=helmert {} +convention=position_vector 
  +step +inv +proj=cart +ellps=evrst30 
  +step +proj=pop +v_3  
  +step +proj=utm +zone=47 +ellps=evrst30'''.format( RID_SHIFT )

#############################################################
pnt = (     ('GNSS610347' , 1_539_840.498, 672_933.772  ),
            ('GNSS610348' , 1_539_779.191, 673_034.016  ),
            ('GNSS610361' , 1_532_996.129, 701_923.398  ),
           ( 'GNSS610362', 1_532_910.154, 701_854.806 ),
           ( 'GPS_007' ,   1_676_063.814, 626_858.214  ),
           ( 'GPS_007A',   1_676_172.766, 626_813.837 ),
           ( 'GPS_610321',   1_548_769.191, 739_940.719 ),
           ( 'GPS_610322',   1_548_518.447, 739_786.618 ),
        )
df = pd.DataFrame( pnt, columns=['Name', 'W84_N', 'W84_E' ] )
#############################################################
df

,Name,W84_N,W84_E
0,GNSS610347,1539840.498,672933.772
1,GNSS610348,1539779.191,673034.016
2,GNSS610361,1532996.129,701923.398
3,GNSS610362,1532910.154,701854.806
4,GPS_007,1676063.814,626858.214
5,GPS_007A,1676172.766,626813.837
6,GPS_610321,1548769.191,739940.719
7,GPS_610322,1548518.447,739786.618


In [99]:
print( "Checking'W84_N','W84_E' ")
for i in list(range(0,len(df),2)):
    duo = df.loc[i:i+1]
    DuoGrid_AziDist(duo, 'W84_N','W84_E' )

Checking'W84_N','W84_E' 
GNSS610347->GNSS610348 grid : azi = 121°26′56.5″   dist =    117.505 meter
GNSS610361->GNSS610362 grid : azi = 218°34′59.99″   dist =    109.984 meter
 GPS_007->GPS_007A   grid : azi = 337°50′18.78″   dist =    117.643 meter
GPS_610321->GPS_610322 grid : azi = 211°34′25.84″   dist =    294.312 meter


In [100]:
#########################################################################################
def DoTransf( row ):
    tr1 = Transformer.from_pipeline( UTMz47_W84toI75 )
    I75 = list( tr1.itransform( [(row.W84_E,row.W84_N) ]) )[0]
    tr2 = Transformer.from_proj( 'EPSG:32647', 'EPSG:4326' )
    W84ll = list( tr2.itransform( [(row.W84_E,row.W84_N) ] ) ) [0]
    tr3 = Transformer.from_proj( 'EPSG:24047', 'EPSG:4240' )
    I75ll = list( tr3.itransform( [(I75[0],I75[1] )] )) [0]
    return np.concatenate(  [np.array(I75), _dd2DMS( W84ll ), _dd2DMS( I75ll ) ] )
df[['I75_E','I75_N', 'W84Lat','W84Lng' , 'I75Lat', 'I75Lng' ]] = \
                                                             df.apply( DoTransf, axis=1, result_type='expand')
#########################################################################################
for col in df.columns:
    if col[-2]=='_':
        df[col] = df[col].astype(float).round(3)
df 

,Name,W84_N,W84_E,I75_E,I75_N,W84Lat,W84Lng,I75Lat,I75Lng
0,GNSS610347,1539840.498,672933.772,673266.492,1539537.594,13°55′24.38249″,100°36′02.38118″,13°55′18.44136″,100°36′14.21276″
1,GNSS610348,1539779.191,673034.016,673366.736,1539476.287,13°55′22.36571″,100°36′05.70677″,13°55′16.42437″,100°36′17.53875″
2,GNSS610361,1532996.129,701923.398,702256.082,1532693.208,13°51′34.83379″,100°52′06.23011″,13°51′28.86118″,100°52′18.18065″
3,GNSS610362,1532910.154,701854.806,702187.490,1532607.233,13°51′32.0541″,100°52′03.92375″,13°51′26.0813″,100°52′15.87396″
4,GPS_007,1676063.814,626858.214,627191.057,1675760.836,15°09′26.20829″,100°10′50.84164″,15°09′20.65008″,100°11′02.54581″
5,GPS_007A,1676172.766,626813.837,627146.680,1675869.788,15°09′29.76148″,100°10′49.37452″,15°09′24.20359″,100°11′01.07856″
6,GPS_610321,1548769.191,739940.719,740273.379,1548466.216,13°59′57.30767″,101°13′16.89968″,13°59′51.35788″,101°13′29.01819″
7,GPS_610322,1548518.447,739786.618,740119.278,1548215.473,13°59′49.19884″,101°13′11.68812″,13°59′43.24847″,101°13′23.80585″


In [101]:
print( "Checking 'I75_N','I75_E' ")
for i in list(range(0,len(df),2)):
    duo = df.loc[i:i+1]
    #import pdb; pdb.set_trace()
    DuoGrid_AziDist(duo, 'I75_N','I75_E' )

Checking 'I75_N','I75_E' 
GNSS610347->GNSS610348 grid : azi = 121°26′56.5″   dist =    117.505 meter
GNSS610361->GNSS610362 grid : azi = 218°34′59.99″   dist =    109.984 meter
 GPS_007->GPS_007A   grid : azi = 337°50′18.78″   dist =    117.643 meter
GPS_610321->GPS_610322 grid : azi = 211°34′26.21″   dist =    294.311 meter
